In [46]:
!pip install ortools

#**Modelo de CP para Orchestra rehearsal**

In [0]:
from ortools.sat.python import cp_model
import time

def OrehearsalSolver(tabpxc, arrcxd, heuvar, heuval):
  model = cp_model.CpModel()

  #**Variables y dominios**

  tabcxo = [] #Tabla Numero de composicion x Orden de composición (1 si la composicion se da en ese orden y 0 si no)
  for i in range(len(tabpxc[0])): 
    fila = [] 
    for j in range(len(tabpxc[0])):
      fila += [model.NewBoolVar('x'+str(i)+str(j))]
    tabcxo += [fila]

  waiti = model.NewIntVar(0, sum(arrcxd)*5, 'z') #Tiempo total de espera

  arrcxo = [] #Arreglo donde los indices son el Numero de la composición y el valor es el orden 
  for i in range(len(tabcxo)):
    arrcxo += [model.NewIntVar(1, 9, 'x'+str(i))]
    for j in range(len(tabcxo)):
      model.Add(arrcxo[i] == j + 1).OnlyEnforceIf(tabcxo[i][j])
      model.Add(arrcxo[i] != j + 1).OnlyEnforceIf(tabcxo[i][j].Not())

  tabpro = [] #Tabla Musico x orden de Composición (Tabla de programación del concierto) es 1 si el musico participa de la composicion y 0 si no
  for i in range(len(tabpxc)): 
    fila = []
    for j in range(len(tabpxc[0])):
      fila += [model.NewBoolVar('pro'+str(i)+str(j))]
    tabpro += [fila]

  tabprowait = [] #Tabla Musico x Composición de espera (indica el tiempo que debe esperar el musico en una composición si no participa en ella)
  for i in range(len(tabpxc)): 
    fila = []
    for j in range(len(tabpxc[0])):
      fila += [model.NewIntVar(0, 10, 'prowait'+str(i)+str(j))]
    tabprowait += [fila]

  tabwait = [] #Tabla Musico x Composicion de espera considerando tiempo de llegada y salida
  #indica el tiempo que debe esperar el musico en una composición si no participa en ella considerando tiempo de llegada y salida
  for i in range(len(tabpxc)): 
    fila = []
    for j in range(len(tabpxc[0])):
      fila += [model.NewIntVar(0, 10, 'wait'+str(i)+str(j))]
    tabwait += [fila]

  tabproalt1 = [] #Tabla de programación alternativa 1 (0 si el musico todavia no llega cuando se conduce cierta composicion y 1 si ya habia llegado)
  for i in range(len(tabpxc)): 
    fila = []
    for j in range(len(tabpxc[0])):
      fila += [model.NewBoolVar('proalt1'+str(i)+str(j))]
    tabproalt1 += [fila]

  tabproalt2 = [] #Tabla de programación alternativa 2 (0 si el musico ya se fue cuando se conduce cierta composicion y 1 si todavia no se va o no llega)
  for i in range(len(tabpxc)): 
    fila = []
    for j in range(len(tabpxc[0])):
      fila += [model.NewBoolVar('proalt2'+str(i)+str(j))]
    tabproalt2 += [fila]

  tabproalt3 = [] #Tabla de programación alternativa 3 (0 si el musico no esta cuando se conduce cierta composicion y 1 si sí esta dicho musico)
  for i in range(len(tabpxc)): 
    fila = []
    for j in range(len(tabpxc[0])):
      fila += [model.NewBoolVar('proalt3'+str(i)+str(j))]
    tabproalt3 += [fila]

  abools = [] #Variables booleanas de restricción para relacionar Tabla de programación con Tabla alternativa 1
  for i in range(len(tabpxc)):
    fila = []
    for j in range(len(tabpxc[0])):
      fila += [model.NewBoolVar('a'+str(i)+str(j))]
    abools += [fila]
  bbools = [] #Variables booleanas de restricción para relacionar Tabla de programación con Tabla alternativa 2
  for i in range(len(tabpxc)):
    fila = []
    for j in range(len(tabpxc[0])):
      fila += [model.NewBoolVar('b'+str(i)+str(j))]
    bbools += [fila]
    
  #**Restricciones**

  model.AddAllDifferent(arrcxo) #El orden de cada composición debe ser distinto
  model.Add(arrcxo[1] < arrcxo[7]) #La composición 2 debe estar antes que la composición 8
  model.Add(arrcxo[4]+1 == arrcxo[5]) #La composición 6 debe darse justo despues de la 5
  
  wait = []
  for i in range(len(tabpxc)):
    for j in range(len(tabpxc[0])):
      orde = []
      iord = []
      for k in range(len(tabcxo)):
        invv = (tabpxc[i][k] - 1)*-1
        orde += [invv*tabcxo[k][j]*arrcxd[k]] #Filtrando para obtener los tiempos de espera de las composiciones en la que un musico no participara
        iord += [tabpxc[i][k]*tabcxo[k][j]] #Filtrando para saber en que composiciones participara cada musico
      model.Add(tabprowait[i][j] == sum(orde)) #Llenando tabla musico x composición de Espera
      model.Add(tabpro[i][j] == sum(iord)) #Llenando tabla de programación
    for j in range(len(tabpro[i])):
      model.Add(tabpro[i][j] + sum(tabpro[i][0:j]) > 0).OnlyEnforceIf(abools[i][j])
      model.Add(tabpro[i][j] + sum(tabpro[i][0:j]) <= 0).OnlyEnforceIf(abools[i][j].Not())
      model.Add(tabproalt1[i][j] == 1).OnlyEnforceIf(abools[i][j]) #Llenando Tabla alternantiva 1
      model.Add(tabproalt1[i][j] != 1).OnlyEnforceIf(abools[i][j].Not())
      model.Add(tabpro[i][len(tabpro[i])-1-j] + sum(tabpro[i][len(tabpro[i])-j:len(tabpro[i])]) > 0).OnlyEnforceIf(bbools[i][j])
      model.Add(tabpro[i][len(tabpro[i])-1-j] + sum(tabpro[i][len(tabpro[i])-j:len(tabpro[i])]) <= 0).OnlyEnforceIf(bbools[i][j].Not())
      model.Add(tabproalt2[i][len(tabpro[i])-1-j] == 1).OnlyEnforceIf(bbools[i][j]) #Llenando Tabla alternativa 2
      model.Add(tabproalt2[i][len(tabpro[i])-1-j] != 1).OnlyEnforceIf(bbools[i][j].Not())
      model.AddMultiplicationEquality(tabproalt3[i][j],[tabproalt1[i][j],tabproalt2[i][j]]) #Llenando tabla de programación alternativa 3
      model.AddMultiplicationEquality(tabwait[i][j],[tabproalt3[i][j],tabprowait[i][j]]) #Llenando Tabla de Espera considerando tiempo de llegada y salida
    wait += [sum(tabwait[i])]
  model.Add(sum(wait) == waiti) #Determinando tiempo total de espera
  
  #Eliminando Simetria
  model.Add(arrcxo[0] <= len(tabpro[0])//2)
  
  #**Optimizador**
  model.Minimize(waiti) #Minimizando tiempo total de espera
  
  #**Heuristicas**  
  model.AddDecisionStrategy(arrcxo, heuvar, heuval) 
  for i in range(len(tabcxo)): model.AddDecisionStrategy(tabcxo[i], heuvar, heuval) 
  for i in range(len(tabpro)): model.AddDecisionStrategy(tabpro[i], heuvar, heuval) 
  for i in range(len(tabprowait)): model.AddDecisionStrategy(tabprowait[i], heuvar, heuval) 
  for i in range(len(tabproalt1)): model.AddDecisionStrategy(tabproalt1[i], heuvar, heuval) 
  for i in range(len(tabproalt2)): model.AddDecisionStrategy(tabproalt2[i], heuvar, heuval)
  for i in range(len(tabproalt3)): model.AddDecisionStrategy(tabproalt3[i], heuvar, heuval)  
  for i in range(len(tabwait)): model.AddDecisionStrategy(tabwait[i], heuvar, heuval) 
  for i in range(len(abools)): model.AddDecisionStrategy(abools[i], heuvar, heuval) 
  for i in range(len(bbools)): model.AddDecisionStrategy(bbools[i], heuvar, heuval) 
 
  #**Solucionador**
  solver = cp_model.CpSolver()
  status = solver.Solve(model)
  print(solver.StatusName(status))
  print(solver.ObjectiveValue())
  if status == cp_model.OPTIMAL:
    #Pintanto tabla de Numero de composicion x Orden de composición
    print("**Tabla Numero de composicion x Orden de composición**")
    print(" ", end=" ")
    for i in range(len(tabcxo)):
      print(i+1, end=" ")
    print()
    for i in range(len(tabcxo)):
      print(i+1, end=" ")
      for j in range(len(tabcxo)):
        print(solver.Value(tabcxo[i][j]),end=" ")
      print()
    print()
    #Pintando tabla de programación
    print("**Tabla de Programación (Musico x Composición)**")
    print(" ", end=" ")
    for i in range(len(tabcxo)):
      for j in range(len(tabcxo)):
        if (solver.Value(tabcxo[j][i]) == 1):
          print(j+1, end=" ")
    print()
    for i in range(5):
      print(i+1, end=" ")
      for j in range(9):
        print(solver.Value(tabpro[i][j]),end=" ")
      print()
    print()
    #Pintando tabla de Espera considerando tiempo de llegada y salida
    print("**Tabla Musico x Composicion de espera considerando tiempo de llegada y salida**")
    print("Indica lo que esperara cada musico por composición:")
    print(" ", end=" ")
    for i in range(len(tabcxo)):
      for j in range(len(tabcxo)):
        if (solver.Value(tabcxo[j][i]) == 1):
          print(j+1, end=" ")
    print()
    for i in range(5):
      print(i+1, end=" ")
      for j in range(9):
        print(solver.Value(tabwait[i][j]),end=" ")
      print()
    print()
    #Pintando tiempo total de espera de todos los musicos
    print("Tiempo total de espera de todos los musicos (Minimizado):")
    print(solver.Value(waiti))
  

tabpxc = [[1, 0, 1, 1, 0, 1, 1, 0, 1], #Tabla del problema Musico x Numero de composicion
          [1, 0, 1, 1, 1, 1, 0, 1, 0],
          [1, 1, 0, 0, 1, 0, 0, 1, 0],
          [1, 0, 0, 0, 1, 0, 1, 0, 1], 
          [0, 1, 0, 1, 1, 1, 1, 0, 0]]
arrcxd = [2, 4, 1, 3, 3, 2, 5, 7, 6] #Duración de composiciones

##**Aplicando el modelo con ->**
##**Heuristica de valor: menor valor**
##**Heuristica de variable: primera variable**

In [48]:
inicio = time.clock()
OrehearsalSolver(tabpxc, arrcxd, 0, 0)
delta_clock = time.clock() - inicio
print()
print("Tiempo de ejecución (segundos): ",delta_clock)

OPTIMAL
16.0
**Tabla Numero de composicion x Orden de composición**
  1 2 3 4 5 6 7 8 9 
1 0 0 1 0 0 0 0 0 0 
2 0 0 0 0 0 0 0 1 0 
3 0 0 0 0 0 0 1 0 0 
4 0 0 0 0 0 1 0 0 0 
5 0 0 0 1 0 0 0 0 0 
6 0 0 0 0 1 0 0 0 0 
7 0 1 0 0 0 0 0 0 0 
8 0 0 0 0 0 0 0 0 1 
9 1 0 0 0 0 0 0 0 0 

**Tabla de Programación (Musico x Composición)**
  9 7 1 5 6 4 3 2 8 
1 1 1 1 0 1 1 1 0 0 
2 0 0 1 1 1 1 1 0 1 
3 0 0 1 1 0 0 0 1 1 
4 1 1 1 1 0 0 0 0 0 
5 0 1 0 1 1 1 0 1 0 

**Tabla Musico x Composicion de espera considerando tiempo de llegada y salida**
Indica lo que esperara cada musico por composición:
  9 7 1 5 6 4 3 2 8 
1 0 0 0 3 0 0 0 0 0 
2 0 0 0 0 0 0 0 4 0 
3 0 0 0 0 2 3 1 0 0 
4 0 0 0 0 0 0 0 0 0 
5 0 0 2 0 0 0 1 0 0 

Tiempo total de espera de todos los musicos (Minimizado):
16

Tiempo de ejecución (segundos):  0.3139459999999996


##**Aplicando el modelo con ->**
##**Heuristica de valor: menor valor**
##**Heuristica de variable: variable con el valor mínimo más pequeño**

In [49]:
inicio = time.clock()
OrehearsalSolver(tabpxc, arrcxd, 1, 0)
delta_clock = time.clock() - inicio
print()
print("Tiempo de ejecución (segundos): ",delta_clock)

OPTIMAL
16.0
**Tabla Numero de composicion x Orden de composición**
  1 2 3 4 5 6 7 8 9 
1 0 0 1 0 0 0 0 0 0 
2 0 0 0 0 0 0 0 1 0 
3 0 0 0 0 0 0 1 0 0 
4 0 0 0 0 0 1 0 0 0 
5 0 0 0 1 0 0 0 0 0 
6 0 0 0 0 1 0 0 0 0 
7 0 1 0 0 0 0 0 0 0 
8 0 0 0 0 0 0 0 0 1 
9 1 0 0 0 0 0 0 0 0 

**Tabla de Programación (Musico x Composición)**
  9 7 1 5 6 4 3 2 8 
1 1 1 1 0 1 1 1 0 0 
2 0 0 1 1 1 1 1 0 1 
3 0 0 1 1 0 0 0 1 1 
4 1 1 1 1 0 0 0 0 0 
5 0 1 0 1 1 1 0 1 0 

**Tabla Musico x Composicion de espera considerando tiempo de llegada y salida**
Indica lo que esperara cada musico por composición:
  9 7 1 5 6 4 3 2 8 
1 0 0 0 3 0 0 0 0 0 
2 0 0 0 0 0 0 0 4 0 
3 0 0 0 0 2 3 1 0 0 
4 0 0 0 0 0 0 0 0 0 
5 0 0 2 0 0 0 1 0 0 

Tiempo total de espera de todos los musicos (Minimizado):
16

Tiempo de ejecución (segundos):  0.3039140000000007


##**Aplicando el modelo con ->**
##**Heuristica de valor: menor valor**
##**Heuristica de variable: variable con el valor máximo más alto**

In [50]:
inicio = time.clock()
OrehearsalSolver(tabpxc, arrcxd, 2, 0)
delta_clock = time.clock() - inicio
print()
print("Tiempo de ejecución (segundos): ",delta_clock)

OPTIMAL
16.0
**Tabla Numero de composicion x Orden de composición**
  1 2 3 4 5 6 7 8 9 
1 0 0 1 0 0 0 0 0 0 
2 0 0 0 0 0 0 0 1 0 
3 0 0 0 0 0 0 1 0 0 
4 0 0 0 0 0 1 0 0 0 
5 0 0 0 1 0 0 0 0 0 
6 0 0 0 0 1 0 0 0 0 
7 0 1 0 0 0 0 0 0 0 
8 0 0 0 0 0 0 0 0 1 
9 1 0 0 0 0 0 0 0 0 

**Tabla de Programación (Musico x Composición)**
  9 7 1 5 6 4 3 2 8 
1 1 1 1 0 1 1 1 0 0 
2 0 0 1 1 1 1 1 0 1 
3 0 0 1 1 0 0 0 1 1 
4 1 1 1 1 0 0 0 0 0 
5 0 1 0 1 1 1 0 1 0 

**Tabla Musico x Composicion de espera considerando tiempo de llegada y salida**
Indica lo que esperara cada musico por composición:
  9 7 1 5 6 4 3 2 8 
1 0 0 0 3 0 0 0 0 0 
2 0 0 0 0 0 0 0 4 0 
3 0 0 0 0 2 3 1 0 0 
4 0 0 0 0 0 0 0 0 0 
5 0 0 2 0 0 0 1 0 0 

Tiempo total de espera de todos los musicos (Minimizado):
16

Tiempo de ejecución (segundos):  0.298648


##**Aplicando el modelo con ->**
##**Heuristica de valor: mayor valor**
##**Heuristica de variable: primera variable**

In [51]:
inicio = time.clock()
OrehearsalSolver(tabpxc, arrcxd, 0, 1)
delta_clock = time.clock() - inicio
print()
print("Tiempo de ejecución (segundos): ",delta_clock)

OPTIMAL
16.0
**Tabla Numero de composicion x Orden de composición**
  1 2 3 4 5 6 7 8 9 
1 0 0 1 0 0 0 0 0 0 
2 0 0 0 0 0 0 0 1 0 
3 0 0 0 0 0 0 1 0 0 
4 0 0 0 0 0 1 0 0 0 
5 0 0 0 1 0 0 0 0 0 
6 0 0 0 0 1 0 0 0 0 
7 0 1 0 0 0 0 0 0 0 
8 0 0 0 0 0 0 0 0 1 
9 1 0 0 0 0 0 0 0 0 

**Tabla de Programación (Musico x Composición)**
  9 7 1 5 6 4 3 2 8 
1 1 1 1 0 1 1 1 0 0 
2 0 0 1 1 1 1 1 0 1 
3 0 0 1 1 0 0 0 1 1 
4 1 1 1 1 0 0 0 0 0 
5 0 1 0 1 1 1 0 1 0 

**Tabla Musico x Composicion de espera considerando tiempo de llegada y salida**
Indica lo que esperara cada musico por composición:
  9 7 1 5 6 4 3 2 8 
1 0 0 0 3 0 0 0 0 0 
2 0 0 0 0 0 0 0 4 0 
3 0 0 0 0 2 3 1 0 0 
4 0 0 0 0 0 0 0 0 0 
5 0 0 2 0 0 0 1 0 0 

Tiempo total de espera de todos los musicos (Minimizado):
16

Tiempo de ejecución (segundos):  0.3063830000000003


##**Aplicando el modelo con ->**
##**Heuristica de valor: mayor valor**
##**Heuristica de variable: variable con el valor mínimo más pequeño**

In [52]:
inicio = time.clock()
OrehearsalSolver(tabpxc, arrcxd, 1, 1)
delta_clock = time.clock() - inicio
print()
print("Tiempo de ejecución (segundos): ",delta_clock)

OPTIMAL
16.0
**Tabla Numero de composicion x Orden de composición**
  1 2 3 4 5 6 7 8 9 
1 0 0 1 0 0 0 0 0 0 
2 0 0 0 0 0 0 0 1 0 
3 0 0 0 0 0 0 1 0 0 
4 0 0 0 0 0 1 0 0 0 
5 0 0 0 1 0 0 0 0 0 
6 0 0 0 0 1 0 0 0 0 
7 0 1 0 0 0 0 0 0 0 
8 0 0 0 0 0 0 0 0 1 
9 1 0 0 0 0 0 0 0 0 

**Tabla de Programación (Musico x Composición)**
  9 7 1 5 6 4 3 2 8 
1 1 1 1 0 1 1 1 0 0 
2 0 0 1 1 1 1 1 0 1 
3 0 0 1 1 0 0 0 1 1 
4 1 1 1 1 0 0 0 0 0 
5 0 1 0 1 1 1 0 1 0 

**Tabla Musico x Composicion de espera considerando tiempo de llegada y salida**
Indica lo que esperara cada musico por composición:
  9 7 1 5 6 4 3 2 8 
1 0 0 0 3 0 0 0 0 0 
2 0 0 0 0 0 0 0 4 0 
3 0 0 0 0 2 3 1 0 0 
4 0 0 0 0 0 0 0 0 0 
5 0 0 2 0 0 0 1 0 0 

Tiempo total de espera de todos los musicos (Minimizado):
16

Tiempo de ejecución (segundos):  0.27939500000000095


##**Aplicando el modelo con ->**
##**Heuristica de valor: mayor valor**
##**Heuristica de variable: variable con el valor máximo más alto**

In [53]:
inicio = time.clock()
OrehearsalSolver(tabpxc, arrcxd, 2, 1)
delta_clock = time.clock() - inicio
print()
print("Tiempo de ejecución (segundos): ",delta_clock)

OPTIMAL
16.0
**Tabla Numero de composicion x Orden de composición**
  1 2 3 4 5 6 7 8 9 
1 0 0 1 0 0 0 0 0 0 
2 0 0 0 0 0 0 0 1 0 
3 0 0 0 0 0 0 1 0 0 
4 0 0 0 0 0 1 0 0 0 
5 0 0 0 1 0 0 0 0 0 
6 0 0 0 0 1 0 0 0 0 
7 0 1 0 0 0 0 0 0 0 
8 0 0 0 0 0 0 0 0 1 
9 1 0 0 0 0 0 0 0 0 

**Tabla de Programación (Musico x Composición)**
  9 7 1 5 6 4 3 2 8 
1 1 1 1 0 1 1 1 0 0 
2 0 0 1 1 1 1 1 0 1 
3 0 0 1 1 0 0 0 1 1 
4 1 1 1 1 0 0 0 0 0 
5 0 1 0 1 1 1 0 1 0 

**Tabla Musico x Composicion de espera considerando tiempo de llegada y salida**
Indica lo que esperara cada musico por composición:
  9 7 1 5 6 4 3 2 8 
1 0 0 0 3 0 0 0 0 0 
2 0 0 0 0 0 0 0 4 0 
3 0 0 0 0 2 3 1 0 0 
4 0 0 0 0 0 0 0 0 0 
5 0 0 2 0 0 0 1 0 0 

Tiempo total de espera de todos los musicos (Minimizado):
16

Tiempo de ejecución (segundos):  0.2913180000000004


##**Aplicando el modelo con ->**
##**Heuristica de valor: Divide el dominio en dos alrededor del centro, y obliga a la variable a tomar el primer valor en la mitad inferior.**
##**Heuristica de variable: primera variable**

In [54]:
inicio = time.clock()
OrehearsalSolver(tabpxc, arrcxd, 0, 2)
delta_clock = time.clock() - inicio
print()
print("Tiempo de ejecución (segundos): ",delta_clock)

OPTIMAL
16.0
**Tabla Numero de composicion x Orden de composición**
  1 2 3 4 5 6 7 8 9 
1 0 0 1 0 0 0 0 0 0 
2 0 0 0 0 0 0 0 1 0 
3 0 0 0 0 0 0 1 0 0 
4 0 0 0 0 0 1 0 0 0 
5 0 0 0 1 0 0 0 0 0 
6 0 0 0 0 1 0 0 0 0 
7 0 1 0 0 0 0 0 0 0 
8 0 0 0 0 0 0 0 0 1 
9 1 0 0 0 0 0 0 0 0 

**Tabla de Programación (Musico x Composición)**
  9 7 1 5 6 4 3 2 8 
1 1 1 1 0 1 1 1 0 0 
2 0 0 1 1 1 1 1 0 1 
3 0 0 1 1 0 0 0 1 1 
4 1 1 1 1 0 0 0 0 0 
5 0 1 0 1 1 1 0 1 0 

**Tabla Musico x Composicion de espera considerando tiempo de llegada y salida**
Indica lo que esperara cada musico por composición:
  9 7 1 5 6 4 3 2 8 
1 0 0 0 3 0 0 0 0 0 
2 0 0 0 0 0 0 0 4 0 
3 0 0 0 0 2 3 1 0 0 
4 0 0 0 0 0 0 0 0 0 
5 0 0 2 0 0 0 1 0 0 

Tiempo total de espera de todos los musicos (Minimizado):
16

Tiempo de ejecución (segundos):  0.30859000000000236


##**Aplicando el modelo con ->**
##**Heuristica de valor: Divide el dominio en dos alrededor del centro, y obliga a la variable a tomar el primer valor en la mitad inferior.**
##**Heuristica de variable: variable con el valor mínimo más pequeño**

In [55]:
inicio = time.clock()
OrehearsalSolver(tabpxc, arrcxd, 1, 2)
delta_clock = time.clock() - inicio
print()
print("Tiempo de ejecución (segundos): ",delta_clock)

OPTIMAL
16.0
**Tabla Numero de composicion x Orden de composición**
  1 2 3 4 5 6 7 8 9 
1 0 0 1 0 0 0 0 0 0 
2 0 0 0 0 0 0 0 1 0 
3 0 0 0 0 0 0 1 0 0 
4 0 0 0 0 0 1 0 0 0 
5 0 0 0 1 0 0 0 0 0 
6 0 0 0 0 1 0 0 0 0 
7 0 1 0 0 0 0 0 0 0 
8 0 0 0 0 0 0 0 0 1 
9 1 0 0 0 0 0 0 0 0 

**Tabla de Programación (Musico x Composición)**
  9 7 1 5 6 4 3 2 8 
1 1 1 1 0 1 1 1 0 0 
2 0 0 1 1 1 1 1 0 1 
3 0 0 1 1 0 0 0 1 1 
4 1 1 1 1 0 0 0 0 0 
5 0 1 0 1 1 1 0 1 0 

**Tabla Musico x Composicion de espera considerando tiempo de llegada y salida**
Indica lo que esperara cada musico por composición:
  9 7 1 5 6 4 3 2 8 
1 0 0 0 3 0 0 0 0 0 
2 0 0 0 0 0 0 0 4 0 
3 0 0 0 0 2 3 1 0 0 
4 0 0 0 0 0 0 0 0 0 
5 0 0 2 0 0 0 1 0 0 

Tiempo total de espera de todos los musicos (Minimizado):
16

Tiempo de ejecución (segundos):  0.3121149999999986


##**Aplicando el modelo con ->**
##**Heuristica de valor: Divide el dominio en dos alrededor del centro, y obliga a la variable a tomar el primer valor en la mitad inferior.**
##**Heuristica de variable: variable con el valor máximo más alto**

In [56]:
inicio = time.clock()
OrehearsalSolver(tabpxc, arrcxd, 2, 2)
delta_clock = time.clock() - inicio
print()
print("Tiempo de ejecución (segundos): ",delta_clock)

OPTIMAL
16.0
**Tabla Numero de composicion x Orden de composición**
  1 2 3 4 5 6 7 8 9 
1 0 0 1 0 0 0 0 0 0 
2 0 0 0 0 0 0 0 1 0 
3 0 0 0 0 0 0 1 0 0 
4 0 0 0 0 0 1 0 0 0 
5 0 0 0 1 0 0 0 0 0 
6 0 0 0 0 1 0 0 0 0 
7 0 1 0 0 0 0 0 0 0 
8 0 0 0 0 0 0 0 0 1 
9 1 0 0 0 0 0 0 0 0 

**Tabla de Programación (Musico x Composición)**
  9 7 1 5 6 4 3 2 8 
1 1 1 1 0 1 1 1 0 0 
2 0 0 1 1 1 1 1 0 1 
3 0 0 1 1 0 0 0 1 1 
4 1 1 1 1 0 0 0 0 0 
5 0 1 0 1 1 1 0 1 0 

**Tabla Musico x Composicion de espera considerando tiempo de llegada y salida**
Indica lo que esperara cada musico por composición:
  9 7 1 5 6 4 3 2 8 
1 0 0 0 3 0 0 0 0 0 
2 0 0 0 0 0 0 0 4 0 
3 0 0 0 0 2 3 1 0 0 
4 0 0 0 0 0 0 0 0 0 
5 0 0 2 0 0 0 1 0 0 

Tiempo total de espera de todos los musicos (Minimizado):
16

Tiempo de ejecución (segundos):  0.30766000000000204


##**Aplicando el modelo con ->**
##**Heuristica de valor: Divide el dominio en dos alrededor del centro, y obliga a la variable a tomar el primer valor en la mitad superior.**
##**Heuristica de variable: primera variable**

In [57]:
inicio = time.clock()
OrehearsalSolver(tabpxc, arrcxd, 0, 3)
delta_clock = time.clock() - inicio
print()
print("Tiempo de ejecución (segundos): ",delta_clock)

OPTIMAL
16.0
**Tabla Numero de composicion x Orden de composición**
  1 2 3 4 5 6 7 8 9 
1 0 0 1 0 0 0 0 0 0 
2 0 0 0 0 0 0 0 1 0 
3 0 0 0 0 0 0 1 0 0 
4 0 0 0 0 0 1 0 0 0 
5 0 0 0 1 0 0 0 0 0 
6 0 0 0 0 1 0 0 0 0 
7 0 1 0 0 0 0 0 0 0 
8 0 0 0 0 0 0 0 0 1 
9 1 0 0 0 0 0 0 0 0 

**Tabla de Programación (Musico x Composición)**
  9 7 1 5 6 4 3 2 8 
1 1 1 1 0 1 1 1 0 0 
2 0 0 1 1 1 1 1 0 1 
3 0 0 1 1 0 0 0 1 1 
4 1 1 1 1 0 0 0 0 0 
5 0 1 0 1 1 1 0 1 0 

**Tabla Musico x Composicion de espera considerando tiempo de llegada y salida**
Indica lo que esperara cada musico por composición:
  9 7 1 5 6 4 3 2 8 
1 0 0 0 3 0 0 0 0 0 
2 0 0 0 0 0 0 0 4 0 
3 0 0 0 0 2 3 1 0 0 
4 0 0 0 0 0 0 0 0 0 
5 0 0 2 0 0 0 1 0 0 

Tiempo total de espera de todos los musicos (Minimizado):
16

Tiempo de ejecución (segundos):  0.31386499999999984


##**Aplicando el modelo con ->**
##**Heuristica de valor: Divide el dominio en dos alrededor del centro, y obliga a la variable a tomar el primer valor en la mitad superior.** 
##**Heuristica de variable: variable con el valor mínimo más pequeño**

In [58]:
inicio = time.clock()
OrehearsalSolver(tabpxc, arrcxd, 1, 3)
delta_clock = time.clock() - inicio
print()
print("Tiempo de ejecución (segundos): ",delta_clock)

OPTIMAL
16.0
**Tabla Numero de composicion x Orden de composición**
  1 2 3 4 5 6 7 8 9 
1 0 0 1 0 0 0 0 0 0 
2 0 0 0 0 0 0 0 1 0 
3 0 0 0 0 0 0 1 0 0 
4 0 0 0 0 0 1 0 0 0 
5 0 0 0 1 0 0 0 0 0 
6 0 0 0 0 1 0 0 0 0 
7 0 1 0 0 0 0 0 0 0 
8 0 0 0 0 0 0 0 0 1 
9 1 0 0 0 0 0 0 0 0 

**Tabla de Programación (Musico x Composición)**
  9 7 1 5 6 4 3 2 8 
1 1 1 1 0 1 1 1 0 0 
2 0 0 1 1 1 1 1 0 1 
3 0 0 1 1 0 0 0 1 1 
4 1 1 1 1 0 0 0 0 0 
5 0 1 0 1 1 1 0 1 0 

**Tabla Musico x Composicion de espera considerando tiempo de llegada y salida**
Indica lo que esperara cada musico por composición:
  9 7 1 5 6 4 3 2 8 
1 0 0 0 3 0 0 0 0 0 
2 0 0 0 0 0 0 0 4 0 
3 0 0 0 0 2 3 1 0 0 
4 0 0 0 0 0 0 0 0 0 
5 0 0 2 0 0 0 1 0 0 

Tiempo total de espera de todos los musicos (Minimizado):
16

Tiempo de ejecución (segundos):  0.3077229999999993


##**Aplicando el modelo con ->**
##**Heuristica de valor: Divide el dominio en dos alrededor del centro, y obliga a la variable a tomar el primer valor en la mitad superior.**
##**Heuristica de variable: variable con el valor máximo más alto**

In [59]:
inicio = time.clock()
OrehearsalSolver(tabpxc, arrcxd, 2, 3)
delta_clock = time.clock() - inicio
print()
print("Tiempo de ejecución (segundos): ",delta_clock)

OPTIMAL
16.0
**Tabla Numero de composicion x Orden de composición**
  1 2 3 4 5 6 7 8 9 
1 0 0 1 0 0 0 0 0 0 
2 0 0 0 0 0 0 0 1 0 
3 0 0 0 0 0 0 1 0 0 
4 0 0 0 0 0 1 0 0 0 
5 0 0 0 1 0 0 0 0 0 
6 0 0 0 0 1 0 0 0 0 
7 0 1 0 0 0 0 0 0 0 
8 0 0 0 0 0 0 0 0 1 
9 1 0 0 0 0 0 0 0 0 

**Tabla de Programación (Musico x Composición)**
  9 7 1 5 6 4 3 2 8 
1 1 1 1 0 1 1 1 0 0 
2 0 0 1 1 1 1 1 0 1 
3 0 0 1 1 0 0 0 1 1 
4 1 1 1 1 0 0 0 0 0 
5 0 1 0 1 1 1 0 1 0 

**Tabla Musico x Composicion de espera considerando tiempo de llegada y salida**
Indica lo que esperara cada musico por composición:
  9 7 1 5 6 4 3 2 8 
1 0 0 0 3 0 0 0 0 0 
2 0 0 0 0 0 0 0 4 0 
3 0 0 0 0 2 3 1 0 0 
4 0 0 0 0 0 0 0 0 0 
5 0 0 2 0 0 0 1 0 0 

Tiempo total de espera de todos los musicos (Minimizado):
16

Tiempo de ejecución (segundos):  0.2970480000000002
